In [17]:
import sys
import os
import time
import logging
import argparse
import numpy as np
from datetime import datetime
# from joblib import Parallel, delayed
import h5py
# Set the parent directory. This assumes your notebook is in the project root or a subdirectory.
# Adjust the path as necessary.
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Custom imports from your project
from estimators.statistical_descriptor import Nagler_WS
from utils.dataset_management import parse_pipeline
from utils.dataset_load import shuffle_data, DatasetLoader
from utils.fold_management import FoldManagement
from utils.label_management import LabelManagement
from utils.balance_management import BalanceManagement
from utils.figures import *
from utils.files_management import *
from utils.dask_chunk import *


In [22]:
model = joblib.load("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_4/group_0/models/KNN_direct/KNN_direct_fold1.joblib")
model

/home/listic/miniconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.5.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/listic/miniconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.5.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Pipeline(memory='.cache',
         steps=[('BandSelector',
                 BandSelector(bands=[0, 1, 2, 3, 4, 5, 6, 7, 8])),
                ('BandTransformer',
                 BandTransformer(bands=[0, 1, 2, 3], transformations=[])),
                ('Hist_SAR', Hist_SAR()),
                ('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=50))],
         verbose=True)

In [38]:
from osgeo import gdal 
dataset = gdal.Open('/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/validation/Acs_GR_jan2jun21/GRANDES-ROUSSES_20210112.tif')

image_3d = dataset.ReadAsArray()
image_3d.shape

(9, 2851, 3900)

In [39]:
image_3d = np.transpose(image_3d, (1, 2, 0))
image_3d.shape

(2851, 3900, 9)

In [40]:

# Initialize the transformer with your model and appropriate parameters
transformer = SlidingWindowTransformer(
    estimator=model,
    window_size=15,
    padding=False,
    use_predict_proba=True  # Use predict_proba to get probabilistic values
)

# Transform the 3D image to get a 2D output
result_2d = transformer.transform(image_3d)

# Print the shape of the result to ensure it is 2D
print(result_2d.shape)

# Save or visualize the result as needed

MemoryError: Unable to allocate 83.2 GiB for an array with shape (15, 15, 9, 2837, 3886, 1) and data type float32

In [34]:
result_2d.ravel()

array([0., 0., 0., ..., 0., 0., 0.])

In [35]:
plt.figure(figsize=(10, 8))
sns.heatmap(result_2d, cmap="viridis")
plt.title('Heatmap of the Resulting 2D Predictions')
plt.show()

ValueError: Must pass 2-d input. shape=(3900, 9, 2851)

<Figure size 1000x800 with 0 Axes>